<a href="https://colab.research.google.com/github/young-lu/MovieSentiment/blob/main/RL_for_Movie_Review_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### imports

In [ ]:
!pip install wget
!pip install transformers

import nltk, re, time, random
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from nltk import sent_tokenize as sentence_tokenize
nltk.download('punkt')
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig


import tensorflow as tf
import torch, wget, os
# !wget -c 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz' -O - | tar -xz 
# Get the GPU device name.
device_name = tf.test.gpu_device_name()
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=83f3f7af11778c2052cead9f851d8d273459f846be901d7e32542bbdab6fa9c6
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
     |████████████████████████████████| 1.3MB 12.7MB/s 
     |████████████████████████████████| 1.1MB 47.7MB/s 
     |████████████████████████████████| 890kB 50.1MB/s 
     |████████████████████████████████| 2.9MB 44.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=fbe7f011ea15c887404d1798113cfbd55ce57b387adc8e2aa85970044c57a47f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!wget -c 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz' -O - | tar -xz 
#!tar -xzf 'aclImdb_v1.tar.gz'

--2020-11-19 18:33:48--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>]  80.23M  9.30MB/s    in 7.8s    

2020-11-19 18:33:56 (10.3 MB/s) - written to stdout [84125825/84125825]



### get sentences from reviews

In [ ]:
pDirs = os.listdir('aclImdb/train/pos')
nDirs = os.listdir('aclImdb/train/neg')

dirs = [pDirs,nDirs]

print("positive reviews: {}\nnegative reviews: {}".format(len(pDirs),len(nDirs)))

pSentences = []
nSentences = []
i=0
starttime=time.time()
max=0
max_sent=0
sent_limit = [5000,5000]
for idx, pdir in enumerate(pDirs[:sent_limit[0]]):
  file = open('aclImdb/train/pos/{}'.format(pdir),'r')
  lines = file.readlines()
  file.close()
  tokens = sentence_tokenize(lines[0])
  if len(tokens) > max:
    max = len(tokens)
    print('review {} sentences: {}'.format(idx, max))
    print(len(pSentences))
  for idx, toke in enumerate(tokens):
    """ 
    simple regex to clean the sentences
    https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/
    """
    if len(toke.strip()) == 1:
      # text = re.sub('[.?!]([.?!])','x' ,lines[0])
      newtoke = re.sub()
    else: 
      newtoke = re.sub(r'<[^>]+>','',toke)
      # print(newtoke)
      newtoke = re.sub(r"([?@!.])\\1+",r'\\1',toke)
      # newtoke = re.sub('[^a-zA-Z]', ' ', newtoke)
      # newtoke = re.sub(r'\s+[a-zA-Z]\s+', ' ', newtoke)
      newtoke = re.sub(r'\s+', ' ', newtoke)
      pSentences.append(newtoke)

print("({:.2f})done extracting {} positive sentences".format(time.time()-starttime, len(pSentences)))
starttime=time.time()
max=0
for ndir in nDirs[:sent_limit[1]]:
  file = open('aclImdb/train/neg/{}'.format(ndir),'r')
  lines = file.readlines()
  file.close()
  tokens = sentence_tokenize(lines[0])
  if len(tokens) > max:
    max = len(tokens)
    print('review {} sentences: {}'.format(idx, max))
  for idx, toke in enumerate(tokens):
    """ 
    simple regex to clean the sentences
    https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/
    """
    # html tags
    newtoke = re.sub(r'<[^>]+>','',toke)
    # newtoke = re.sub(r'(?<![.])(?![.]{3})(\p{P})\1',r'$1',toke)
    # print(newtoke)
    newtoke = re.sub(r"([?@!.])\\1+",r'\\1',toke)
    # random single chars
    # newtoke = re.sub('[^a-zA-Z]', ' ', newtoke)
    # newtoke = re.sub(r'\s+[a-zA-Z]\s+', ' ', newtoke)
    newtoke = re.sub(r'\s+', ' ', newtoke)
    nSentences.append(newtoke)

print("({:.2f}) done extracting {} negative sentences".format(time.time() - starttime, len(nSentences)))

# prune sentences:


positive reviews: 12500
negative reviews: 12500
review 0 sentences: 10
0
review 5 sentences: 12
30
review 11 sentences: 19
79
review 12 sentences: 24
98
review 24 sentences: 29
239
review 35 sentences: 41
372
review 56 sentences: 42
611
review 235 sentences: 43
2692
review 390 sentences: 45
4279
review 428 sentences: 53
4739
review 998 sentences: 55
10632
review 1356 sentences: 282
14330
(2.88)done extracting 52696 positive sentences
review 0 sentences: 17
review 1 sentences: 19
review 18 sentences: 25
review 9 sentences: 29
review 14 sentences: 30
review 9 sentences: 32
review 13 sentences: 40
review 7 sentences: 41
review 1 sentences: 60
review 7 sentences: 75
(2.93) done extracting 55447 negative sentences


TypeError: ignored

In [ ]:
## printout random thing

file = open('aclImdb/train/pos/{}'.format(pDirs[1356]),'r')
lines = file.readlines()
file.close()
tokens = sentence_tokenize(lines[0])
print("sentences in this review: {}".format(len(tokens)))
for idx,sent in enumerate(tokens):
  print(idx,'--',sent)

for i in range(14330-10,14330-10+282):
  print(i,pSentences[i])
tokens = lines[0]
for 

# text = re.sub(r'(\s+)','',text)
print(text)

sentences in this review: 282
0 -- Smallville episode Justice is the best episode of Smallville !
1 -- !
2 -- !
3 -- !
4 -- !
5 -- !
6 -- !
7 -- !
8 -- !
9 -- !
10 -- !
11 -- !
12 -- !
13 -- !
14 -- !
15 -- !
16 -- !
17 -- !
18 -- !
19 -- !
20 -- !
21 -- !
22 -- !
23 -- !
24 -- !
25 -- !
26 -- !
27 -- !
28 -- !
29 -- !
30 -- !
31 -- !
32 -- !
33 -- !
34 -- !
35 -- !
36 -- !
37 -- !
38 -- !
39 -- !
40 -- !
41 -- !
42 -- !
43 -- !
44 -- !
45 -- !
46 -- !
47 -- !
48 -- !
49 -- !
50 -- !
51 -- !
52 -- !
53 -- !
54 -- !
55 -- !
56 -- !
57 -- !
58 -- !
59 -- !
60 -- !
61 -- !
62 -- !
63 -- !
64 -- !
65 -- !
66 -- !
67 -- !
68 -- !
69 -- !
70 -- !
71 -- !
72 -- !
73 -- !
74 -- !
75 -- !
76 -- !
77 -- !
78 -- !
79 -- !
80 -- !
81 -- !
82 -- !
83 -- !
84 -- !
85 -- !
86 -- !
87 -- !
88 -- !
89 -- !
90 -- !
91 -- !
92 -- !
93 -- !
94 -- !
95 -- !
96 -- !
97 -- !
98 -- !
99 -- !
100 -- !
101 -- !
102 -- !
103 -- !
104 -- !
105 -- !
106 -- !
107 -- !
108 -- !
109 -- !
110 -- !
111 -- !
112 -- !
11

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

### encode sentences

In [ ]:
sentences = []
encodings = []
labels = []
masks = []
starttime = time.time()
for psent in pSentences:
  encoded_sent = tokenizer.encode(psent,add_special_tokens = True)
  # print(samp.split(' '))
  # print(encodings[0],'\n\t',len(encoded_sent))
  sentences.append(psent)
  encodings.append(encoded_sent)
  labels.append(1)
  # if len(encodings) == 10000:
    # break
print("({:.2f}) positive sentenced encoded.".format(time.time()-starttime))
for nsent in nSentences:
  encoded_sent = tokenizer.encode(nsent,add_special_tokens = True)
  # print(samp.split(' '))
  # print(encodings[0],'\n\t',len(encoded_sent))
  sentences.append(nsent)
  encodings.append(encoded_sent)
  labels.append(0)
  # if len(encodings) == 20000:
    # break
endtime=time.time()
print("negative sentences encoded.")
print("total ({}) sentence encoding time:\t{:.4f}".format(len(encodings),endtime-starttime))


sentence_token_ids = pad_sequences(encodings, maxlen=64, dtype="long", value=0, truncating="post", padding="post")
# starttime=time.time()
# i=0
# for sent in sentence_token_ids:
#   i+=1
#   masks.append([int(toke > 0) for toke in sent])
#   if i % 25000 == 0:
#     print("({:.2f}) masks encoded:\t{}".format(time.time()-starttime,i))
# print("({:.2f}) masks created".format(time.time()-starttime))


Token indices sequence length is longer than the specified maximum sequence length for this model (730 > 512). Running this sequence through the model will result in indexing errors


(29.92) positive sentenced encoded.
negative sentences encoded.
total (108292) sentence encoding time:	60.1005


In [ ]:
max=0
maxInd = None
for idx,sent in enumerate(encodings):
  if len(sent) > max:
    maxInd = idx
    max = len(sent)

print(max,encodings[maxInd])

848 [101, 2144, 2087, 3319, 1005, 1055, 1997, 2023, 2143, 2024, 1997, 11326, 1005, 1055, 2464, 5476, 1005, 1055, 3283, 1045, 1005, 1040, 2066, 2000, 5587, 1037, 2062, 3522, 2028, 1010, 1996, 2143, 2330, 1005, 1055, 2007, 4518, 8333, 1997, 1038, 1011, 2459, 1005, 1055, 8647, 2762, 1010, 1996, 2143, 3013, 1005, 1055, 2000, 28626, 14121, 1005, 1055, 5292, 29441, 5804, 1006, 2952, 1007, 8814, 3367, 2839, 1998, 2010, 14895, 2770, 2005, 3104, 2096, 2437, 2037, 2126, 2000, 8042, 1005, 1055, 11865, 11106, 14544, 15742, 1010, 2320, 2503, 1010, 2027, 2024, 2139, 23736, 27235, 2094, 2011, 15742, 3095, 5073, 1010, 1996, 2143, 2059, 3013, 1005, 1055, 2000, 2028, 1997, 2116, 3034, 3496, 1005, 1055, 2007, 2632, 8428, 15315, 13390, 3228, 1037, 11519, 8605, 1997, 12500, 8042, 8320, 2075, 2010, 2961, 1005, 1055, 2000, 1000, 7209, 3377, 1000, 2096, 14121, 1005, 1055, 2839, 2003, 3491, 2004, 3254, 2746, 2000, 5382, 1996, 15742, 7939, 4697, 2078, 1005, 1055, 2024, 3236, 2039, 1999, 1037, 5913, 2088, 1011, 

In [ ]:
# import bert
# from bert import run_classifier

max_len=128
count=0
max=0
maxInd = None
for idx,sent in enumerate(encodings):
  if len(sent) > max:
    maxInd = idx
    max = len(sent)
  if len(sent) > max_len:
    encodings[idx] = sent[:max_len]
    encodings[idx][-1] = 102
    count+=1
print("sentences shortened: {}".format(count))
print(max,encodings[maxInd])

inputs = {
    "data": sentences[26],
    "mask": masks[26],
    "labels": labels[26] 
}

# train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
#                                                                    text_a = x['data'], 
#                                                                    text_b = None, 
#                                                                    label = x['labels']), axis = 1

NameError: ignored

In [ ]:
sentence_token_ids = pad_sequences(encodings, maxlen=64, dtype="long", value=0, truncating="post", padding="post")
starttime=time.time()
i=0
for sent in sentence_token_ids:
  i+=1
  masks.append([int(toke > 0) for toke in sent])
  if i % 25000 == 0:
    print("({:.2f}) masks encoded:\t{}".format(time.time()-starttime,i))
print("({:.2f}) masks created".format(time.time()-starttime))

(1.22) masks encoded:	25000
(2.25) masks encoded:	50000
(3.28) masks encoded:	75000
(4.30) masks encoded:	100000
(4.67) masks created


## Model Config

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(sentence_token_ids, labels, random_state=2020, test_size=0.1)
train_masks, test_masks, _, _ = train_test_split(masks, labels,random_state=2020, test_size=0.1)

train_x = torch.tensor(train_x)
test_x = torch.tensor(test_x)
train_y = torch.tensor(train_y)
test_y = torch.tensor(test_y)
train_masks = torch.tensor(train_masks)
test_masks = torch.tensor(test_masks)

batch_size = 64
epochs=4

train_data = TensorDataset(train_x, train_masks, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_x, test_masks, test_y)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# how to make the model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
# Tell pytorch to run this model on the GPU.
model.cuda()

seed_val = 42
random.seed(seed_val)
np.random.seed()
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


optimizer = AdamW(model.parameters(),lr=2e-5)
                  # lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  # eps = 1e-8 # args.adam_epsilon  - default is 1e-8.

from transformers import get_linear_schedule_with_warmup

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# TF, Keras


## BASIC CLASSIFICATION



In [ ]:
# from nltk import sent_tokenize as sentence_tokenize
# import transformers
# from transformers import BertTokenizer
# from keras.preprocessing.sequence import pad_sequences

In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.3.0
Eager mode:  True
Hub version:  0.10.0
GPU is available


In [ ]:
train_data, test_data = tfds.load('imdb_reviews', split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

"""

dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

"""

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBX39EU/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBX39EU/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBX39EU/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
model = tf.keras.Sequential()
'https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1'
hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1", output_shape=[20], input_shape=[], dtype=tf.string, trainable=True)
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='adam',loss=tf.losses.BinaryCrossentropy(from_logits=True),metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)
results = model.evaluate(test_data, test_labels)

print(results)

Epoch 1/40
30/30 [==============================] - 2s 58ms/step - loss: 0.7434 - accuracy: 0.5263 - val_loss: 0.6633 - val_accuracy: 0.5984
Epoch 2/40
30/30 [==============================] - 2s 53ms/step - loss: 0.6431 - accuracy: 0.6335 - val_loss: 0.6200 - val_accuracy: 0.6558
Epoch 3/40
30/30 [==============================] - 2s 53ms/step - loss: 0.5964 - accuracy: 0.6854 - val_loss: 0.5812 - val_accuracy: 0.7020
Epoch 4/40
30/30 [==============================] - 2s 53ms/step - loss: 0.5518 - accuracy: 0.7329 - val_loss: 0.5427 - val_accuracy: 0.7397
Epoch 5/40
30/30 [==============================] - 2s 52ms/step - loss: 0.5066 - accuracy: 0.7675 - val_loss: 0.5054 - val_accuracy: 0.7648
Epoch 6/40
30/30 [==============================] - 2s 52ms/step - loss: 0.4641 - accuracy: 0.8007 - val_loss: 0.4697 - val_accuracy: 0.7917
Epoch 7/40
30/30 [==============================] - 2s 52ms/step - loss: 0.4224 - accuracy: 0.8262 - val_loss: 0.4388 - val_accuracy: 0.8044
Epoch 8/40
30

## BERT

In [2]:
!pip3 install --quiet tensorflow
!pip3 install --quiet tensorflow_text
!pip install -q tf-models-official


     |████████████████████████████████| 2.6MB 15.0MB/s 
     |████████████████████████████████| 849kB 15.7MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
     |████████████████████████████████| 358kB 25.8MB/s 
     |████████████████████████████████| 174kB 51.5MB/s 
     |████████████████████████████████| 1.1MB 48.3MB/s 
     |████████████████████████████████| 36.7MB 86kB/s 


In [3]:
import os
import shutil

import tensorflow_probability as tfp

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer

# import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [3]:
logs = [.6,.4]
cat = tfp.distributions.Categorical(probs=logs)
acts = (cat.sample(100))
log_probs=[]
for act in acts:
  log_probs.append(cat.log_prob(act))
  # print(act,'-->',cat.log_prob(act))

t = sum(acts)/len(acts)
print("% of .4 prob action:",t.numpy())
print(sum(log_probs))

% of .4 prob action: 0.43
tf.Tensor(-68.51756, shape=(), dtype=float32)


In [4]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
'''
train_data, test_data = tfds.load('imdb_reviews', split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)
'''
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Version:  2.3.0
Eager mode:  True
Hub version:  0.10.0
GPU is available
84131840/84125825 [==============================] - 4s 0us/step


In [5]:
# create the train/validation split
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)


val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)
print(class_names)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
['neg', 'pos']


In [6]:
encoder_url = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
preprocessor_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1'

In [ ]:
# tutorial test
bert_preprocess_model = hub.KerasLayer(preprocessor_url)
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)
print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')


Keys       : ['input_word_ids', 'input_mask', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
for xbatch,ybatch in train_ds.take(1):
  print(xbatch[0])
  st = str(xbatch[0].numpy(),'utf-8')
  # print(st)
  print(bert_preprocess_model([st]))

In [ ]:
# tutorial test
bert_model = hub.KerasLayer(encoder_url)
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {preprocessor_url}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')


In [24]:
# tutorial build the model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer(preprocessor_url, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
# encoder same as bert_model
encoder = hub.KerasLayer(encoder_url, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
net = tf.keras.layers.Dropout(0.1)(net)
net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
classifier_model = tf.keras.Model(text_input, net)

In [ ]:
# text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
# preprocessing_layer = hub.KerasLayer(preprocessor_url, name='preprocessing')
# encoder_inputs = preprocessing_layer(text_input)
# # encoder same as bert_model
# encoder = hub.KerasLayer(encoder_url, trainable=True, name='BERT_encoder')
# outputs = encoder(encoder_inputs)
# out = outputs
# test_model = tf.keras.Model(text_input)


In [ ]:
probs =[.2,.4,.35,.7,.8]
cat = tfp.distributions.Categorical(probs=probs)
print(cat.sample())

cat2 = tf.random.categorical(logits=[tf.nn.softmax(probs)], num_samples=1)
print(cat2)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor([[2]], shape=(1, 1), dtype=int64)


In [ ]:
tf.cast(cat2,float)
print(cat2)

tf.Tensor([[2]], shape=(1, 1), dtype=int64)


In [12]:
text_test = ['I hated this movie so much that it made me happy!']
# Let's check that the model runs with the output of the preprocessing model.
# classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print((bert_raw_result))

tf.Tensor([[-0.34755278]], shape=(1, 1), dtype=float32)


In [25]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# loss = tf.keras.losses.mean_squared_error(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()
init_lr = 3e-5
epochs = 1


steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)


optimizer = optimization.create_optimizer(
    init_lr=init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    optimizer_type='adamw'
)
optim = optimization.create_optimizer(
    init_lr=init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=int(num_train_steps*.13),
    optimizer_type='adamw'
)
classifier_model.compile(optimizer=optim,loss=loss,metrics=metrics)


In [60]:
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

print(steps_per_epoch)
print(num_train_steps)
print(num_warmup_steps)

625
625
62


In [ ]:
for ex,_ in train_ds.take(1):
  print((ex[0].numpy()))

b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />There\'s a mysteri

Train with  ```model.fit()```



In [26]:
print(f'Training model with {encoder_url}')
# tempx = []
# tempy = []
# for xbatch,ybatch in train_ds.take(32):
#   for idx, x in enumerate(xbatch):
#     tempy.append(ybatch[idx].numpy())
#     tempx.append(str(x.numpy(),'utf-8'))
# print(tempx[0])
# print(tempy[0])

# history = classifier_model.fit(
#     x=np.asarray(tempx),
#     y=np.asarray(tempy),
#     validation_split=.2,
#     epochs=5
# )

history = classifier_model.fit(x=train_ds, validation_data=val_ds, epochs=5)



Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/5
625/625 [==============================] - 139s 223ms/step - loss: 0.4613 - binary_accuracy: 0.7614 - val_loss: 0.3780 - val_binary_accuracy: 0.8202
Epoch 2/5
625/625 [==============================] - ETA: 0s - loss: 0.3558 - binary_accuracy: 0.8359

KeyboardInterrupt: ignored

In [43]:
history.history.keys()
print((history.history['binary_accuracy']))

[0.53125]


Test

In [ ]:
test_loss, test_accuracy = classifier_model.evaluate(test_ds)
print(f'Loss: {test_loss}')
print(f'Accuracy: {test_accuracy}')


782/782 [==============================] - 70s 89ms/step - loss: 0.3607 - binary_accuracy: 0.8317
Loss: 0.36073803901672363
Accuracy: 0.8316799998283386


In [ ]:
sample = ['this is a sample sentence']

# classifier_model(sample)
classifier_model(tf.constant(sample))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.75665253]], dtype=float32)>

train manually?

## LSTM

In [31]:
import numpy as np
import sys
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

tfds.disable_progress_bar()

# lstm_dataset, lstm_info = tfds.load('imdb_reviews', with_info=True,
#                           as_supervised=True)

# lstm_train_dataset, lstm_test_dataset = lstm_dataset['train'], lstm_dataset['test']
# lstm_train_dataset.element_spec

# lstm_train_dataset = lstm_train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
# lstm_test_dataset = lstm_test_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

AUTOTUNE = tf.data.experimental.AUTOTUNE
BUFFER_SIZE = 1000 # vocabulary size?
BATCH_SIZE = 64
seed = 43

lstm_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset='training',
    seed=seed)

lstm_train_ds = lstm_train_ds.shuffle(BUFFER_SIZE).cache().prefetch(AUTOTUNE)

lstm_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset='training',
    seed=seed)
lstm_val_ds = lstm_val_ds.cache().prefetch(AUTOTUNE)

lstm_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=BATCH_SIZE)
lstm_test_ds = lstm_test_ds.cache().prefetch(AUTOTUNE)



Version:  2.3.0
Eager mode:  True
Hub version:  0.10.0
GPU is available
Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.


In [14]:
for ex,lab in lstm_train_ds.take(5):
  print("review:",ex.numpy()[0])

review: b"Go immediately and rent this movie. It will be be on a bottom shelf in your local video store and will be covered in dust. No one will have touched it in years. It may even be a $.50 special! It's worth ten bucks, I swear! Buy it! There aren't very many films than can compare with this - the celluloid version of that goo that forms at the bottom of a trash can after a few years. Yes, I gave it a '1,' but it really deserves much lower. 1-10 scales were not designed with stuff like this in mind."
review: b'This may not be one of the best movies ever made but overall it\'s a very enjoyable, light-hearted piece of froth in which everyone involved seems to be having a good time. Highly recommended for it\'s feel-good factor alone. OK, so Frank Sinatra\'s "acting" leaves a lot to be desired but his singing is a great redeeming feature and the songs fit in perfectly with the romantic atmosphere of the film. Sinatra went on to make many more films where his undoubted acting ability s

In [28]:
# encoder for the tutorial LSTM-- it encodes tokens, but we should use the BERT tokens as inputs

VOCAB_SIZE=BUFFER_SIZE

encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(lstm_train_ds.map(lambda text, label: text))


In [15]:
vocab = np.array(encoder.get_vocabulary())
vocab[:15]
print("bytes length",len(ex[0].numpy()))
st = str(ex[0].numpy(),'utf-8')
print("str length",len(st.split(' ')))
encoding = (encoder(ex[0].numpy()))
print("encoding length",len(encoding))

bytes length 751
str length 124
encoding length 124


In [ ]:
# example
print(len(example[0]), example[0])
encoded_example = encoder(example)[:3].numpy()
encoded_example[:,:35]

array([[  1,   5,  61,   2,  18,  23,  69, 104,  45, 319, 287,  16,   4,
          1,  96,   5,   1,  47, 348,   1,   1,  42,   1,  19,  23, 178,
          6,   1, 123,   1,  78,  22, 835,  98,   1],
       [ 11, 238,   5,   1,   1,   7, 192,   6,  95,  23,   1,   9,   1,
         30,   1,  39,   2,   1,   1,  21, 737,   9,   1,  36,   2, 312,
         30, 194,   1,   3, 533,  33, 242,   2,   1],
       [ 48,   7,   4, 169,   5, 653, 574,  46,   5, 356,   1,  13,   4,
        463,   1,  70,  93, 150,   3, 172,  15,   4, 863,  30,   1,  13,
         10, 688,   9,  21,   3, 405, 636, 146,   9]])

In [29]:
# get the vocabulary size for a small test set
# import torch
# from torch.distributions import Categorical

lstm_model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1,activation=None)
])

lstm_model2 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2,activation=None)
])

lstm_optim = tf.optimizers.Adam()

# make function to compute action distribution
# def get_policy(obs):
#     logits = lstm_model(obs)
#     return Categorical(logits=logits)

# # make action selection function (outputs int actions, sampled from policy)
# def get_action(obs):
#     return get_policy(obs).sample().item()

# def compute_loss(obs, act, weights=None):
#   weights = np.ones(len(obs))
#   logp = get_policy(obs).log_prob(act)
#   return -(logp * weights).mean()

# make the LSTM for this vocabulary size


# use the LSTM to show the prob of predicting word classes 
#   (binary, output doesnt matter now)


# train the LSTM for a few epochs with the classification of the sentences, or BERT's ?

# class policy_model:

## LSTM single training epoch

### attmpt_1

In [ ]:
# from tensorflow.compat.v1.distributions import Multinomial as multinomial
'''
tf.random.categorical(logits=np.array([[.1,.2,.9]]),num_samples=1)
'''

import sys

def count_empty(x):
  ct=0
  for each in x:
    if each=='' or each is None:
      print("-->",each)
      ct+=1
  return ct

with tf.GradientTape() as t:

  for example, label in lstm_train_ds.take(1):
    # run through the lstm
    # batch_sentences = example
    batch_logits = lstm_model2(example)

    batch_actions =[[] for _ in batch_logits]
    log_prob_sums =[None for _ in batch_logits]

    for idx,sentence in enumerate(batch_logits):
      temp_log_probs = []
      for probs in sentence:
        # make a distribution for each word
        pol = tfp.distributions.Categorical(probs=tf.nn.softmax(probs))
        # sample from the distribution
        act = pol.sample()
        # save the action (0=keep,1=delete)
        batch_actions[idx].append(act)
        # with that action, get the log_prob
        log_prob = pol.log_prob(act)
        # save that log_prob to a list 
        temp_log_probs.append(log_prob)

      # sum the log probs for this sentence and add to list
      log_prob_sums.append(sum(tf.nn.softmax(temp_log_probs)))

    # split original sentences by space and remove words where action==1
    new_example = [str(ex.numpy(),'utf-8') for ex in example]

    for idx, ex in enumerate(new_example):
      acts = batch_actions[idx]
      splitstr = ex.split(" ")
      
      ''' PROBLEM-- SOMETIMES THERE ARE MORE WORDS THAN ACTIONS '''
      ''' IS THIS AN ISSUE WITH THE PREPROCESSOR? NOT CONSISTENT '''
      if (len(splitstr) > len(acts)):
        print((splitstr[::-1]))
        print((acts[::-1]))
        print(count_empty(splitstr))
        print('{} ERROR: more tokens than actions...'.format(idx))
      for i in range(len(splitstr)):
        if i < len(acts) and acts[i] == 1:
            splitstr[i] = ''
      new_example[idx] = ' '.join(splitstr)

    # convert the sentences to byte-strings ? maybe...

    # get classification results from BERT for new strings
    # training
    print(new_example)
    raw_outs = []
    # for new in new_example:
    #   raw_outs.append(classifier_model(tf.constant([new])))
    raw_outs = classifier_model(tf.constant(new_example))
    # print("raw output length", len(raw_outs))
    print(raw_outs.shape,'\n',raw_outs)
    print('labels',label.shape,'\n',label)
    loss_object = tf.keras.losses.mean_squared_error(raw_outs,tf.cast(label,float))

    # use BERT's loss to 
    print(loss_object)

    # use the negative losses to weight the log_probs
    ''' now-- how do I use these in the update? '''
    for idx, loss in enumerate(loss_object):
      log_prob_sums[idx] *= -(loss)

    sys.exit()
    # get classification results from BERT for original strings
    # evaluating

    # get weights for the words
    # higher weights 

    # policies = np.array([tfp.distributions.ategorical(probs=tf.nn.log_softmax(sentence)) for sentence in batch_logits])
    # policies = [tf.random.categorical(logits=(sentence),num_samples=1) for sentence in batch_logits]
    # print(len(batch_logits[0]))
    # stringarr = (str(example[0].numpy(),'utf-8').split(' '))
    # print(len(stringarr))
    # print(sum([len(toke) for toke in stringarr]))
    # print(batch_logits[0][:70])
  sys.exit()

# make update to LSTM optimizer

'''
  # diff is the baseline
  # logsum = tf.reduce_mean(logsum_batch * weight)
  logsum = tf.reduce_sum(tf.nn.log_softmax(batch_logits[0]))
  # y = logsum * weight

# print(st==base)
# def compute_loss():
  # retrun -(classifier_model(base) - classifier_model(x))

# lstm_optim.minimize(compute_loss,var_list = tf.constant([]))


gradients = t.gradient(logsum, lstm_model.trainable_variables)
# The optimize applies the update, using the variables
# and the optimizer update rule
# opt.apply_gradients(zip(gradients, trainable_variables))
print(gradients)
if gradients:
  lstm_optim.apply_gradients(zip(gradients,lstm_model.trainable_variables))
else:
  print('ERROR: no grads')
# LSTM reward is BERT_original_sentence - BERT_new_sentence
#   these are the difference in predictions between original and old
'''

['I\'ll       like this doesn\'t  a full review.<br /><br />Given  setting,   could  been   special.        later" or  a      first      the movie  was really  directer  a decent   cinematography and   I don\'t  He  to capture true   an empty  Underground.<br /><br />Characters  a  let   "heroine" in this movie  a   of crap, and you really  care    or not. As   have  before,      homeless people and    who managed   me a   two(whether  of the writers   /><br   villain,  kept in the   the first  of the movie, but when  is revealed   really  I   it  lets just      sister      a wrestling    in all  is just another mediocre   which falls  the trap of     formula. This  had     but    hit the mark.<br  />Just    lame    the characters  this movie had   FIVE   finish  and kill  main  INSTEAD   AWAY.', "This Movie is complete crap! Avoid  waste  celluloid  all  it  rambling  incoherent. I pride myself      70's  cinema from   Salo to  Kitty. I like    I  a    if watching horses      film    

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### attmpt_2

In [33]:
# from tensorflow.compat.v1.distributions import Multinomial as multinomial
'''
tf.random.categorical(logits=np.array([[.1,.2,.9]]),num_samples=1)
'''

import sys
import time

def count_empty(x):
  ct=0
  for each in x:
    if each=='' or each is None:
      print("-->",each)
      ct+=1
  return ct

def step(model, X):
  # return batch_logits, new_sentences, and log_prob_sums

  batch_logits = model(X)
  batch_actions =[[] for _ in batch_logits]
  log_prob_sums =[None for _ in batch_logits]

  for idx,sentence in enumerate(batch_logits):
    temp_log_probs = []
    for probs in sentence:
      # make a distribution for each word
      pol = tfp.distributions.Categorical(probs=tf.nn.softmax(probs))
      # sample from the distribution
      act = pol.sample()
      # save the action (0=keep,1=delete)
      batch_actions[idx].append(act)
      # with that action, get the log_prob
      log_prob = pol.log_prob(act)
      # save that log_prob to a list 
      temp_log_probs.append(log_prob)

    # sum the log probs for this sentence and add to list
    log_prob_sums[idx] = sum(temp_log_probs)

  # split original sentences by space and remove words where action==1
  new_X = [str(ex.numpy(),'utf-8') for ex in X]

  for idx,ex in enumerate(new_X):
    acts = batch_actions[idx]
    splitstr = ex.split(" ")

    ''' PROBLEM-- SOMETIMES THERE ARE MORE WORDS THAN ACTIONS '''
    ''' IS THIS AN ISSUE WITH THE PREPROCESSOR? NOT CONSISTENT '''
    if (len(splitstr) > len(acts)):
      print((splitstr[::-1]))
      print((acts[::-1]))
      print(count_empty(splitstr))
      print('{} ERROR: more tokens than actions...'.format(idx))
    for i in range(len(splitstr)):
      if i < len(acts) and acts[i] == 1:
        splitstr[i] = ''
    new_X[idx] = ' '.join(splitstr)
  return batch_logits, new_X, log_prob_sums


'''
test step()
'''

for example, label in lstm_train_ds.take(10):
  # run through the lstm
  # batch_sentences = example
  with tf.GradientTape() as t:
    start = time.time()
    logits, new_example, logprob_sums = step(lstm_model2, example)
    print('model output time: ',time.time()-start)
    print(len(logits),'total logits')
    # print(new_example[0])
    print('logprobs')
    print(len(logprob_sums), logprob_sums)
    # print(logits[0])
  #   batch_logits = lstm_model2(example)
  #   batch_actions =[[] for _ in batch_logits]
  #   log_prob_sums =[None for _ in batch_logits]
  #   for idx,sentence in enumerate(batch_logits):
  #     temp_log_probs = []
  #     for probs in sentence:
  #       pol = tfp.distributions.Categorical(probs=tf.nn.softmax(probs))
  #       act = pol.sample()
  #       batch_actions[idx].append(act)
  #       log_prob = pol.log_prob(act)
  #       temp_log_probs.append(log_prob)
  #     log_prob_sums.append(sum(tf.nn.softmax(temp_log_probs)))
  # new_example = [str(ex.numpy(),'utf-8') for ex in example]
  # for idx, ex in enumerate(new_example):
  #   acts = batch_actions[idx]
  #   splitstr = ex.split(" ")
  #   if (len(splitstr) > len(acts)):
  #     print('{} ERROR: more tokens than actions...'.format(idx))
  #   for i in range(len(splitstr)):
  #     if i < len(acts) and acts[i] == 1:
  #         splitstr[i] = ''
  #   new_example[idx] = ' '.join(splitstr)
    # print(new_example)
    raw_outs = []
    # need to FIT bert on these
    raw_outs = classifier_model(tf.constant(new_example))
    print(raw_outs.shape,'\n',raw_outs)
    print('labels',label.shape,'\n',label)
    loss_object = tf.keras.losses.mean_squared_error(raw_outs,tf.cast(label,float))
    print('loss object:', loss_object)
    # use BERT's loss for update:
    # use the negative losses to weight the log_probs
    ''' now-- how do I use these in the update? '''
    # for idx, loss in enumerate(loss_object):
    #   if loss is not None:
    #     logprob_sums[idx] *= -(loss)
    weighted_logprobs = -(loss_object)*logprob_sums
    print('logprob_sums:\n\t', weighted_logprobs)
      
  # get gradients and apply update
  # gradients = t.gradient(logprob_sums, lstm_model2.trainable_variables)
  gradients = t.gradient(weighted_logprobs, lstm_model2.trainable_variables)
  print('gradients[0]:', gradients[0])
  lstm_optim.apply_gradients(zip(gradients,lstm_model2.trainable_variables))
  print('optimizer step success.')
  # sys.exit()

  '''
  end for ex,label in ....
  '''

  # policies = np.array([tfp.distributions.ategorical(probs=tf.nn.log_softmax(sentence)) for sentence in batch_logits])
  # policies = [tf.random.categorical(logits=(sentence),num_samples=1) for sentence in batch_logits]
  # print(len(batch_logits[0]))
  # stringarr = (str(example[0].numpy(),'utf-8').split(' '))
  # print(len(stringarr))
  # print(sum([len(toke) for toke in stringarr]))
  # print(batch_logits[0][:70])
print('DONE.')

# make update to LSTM optimizer

'''
# diff is the baseline
# logsum = tf.reduce_mean(logsum_batch * weight)
logsum = tf.reduce_sum(tf.nn.log_softmax(batch_logits[0]))
# y = logsum * weight

# print(st==base)
# def compute_loss():
# retrun -(classifier_model(base) - classifier_model(x))

# lstm_optim.minimize(compute_loss,var_list = tf.constant([]))


gradients = t.gradient(logsum, lstm_model.trainable_variables)
# The optimize applies the update, using the variables
# and the optimizer update rule
# opt.apply_gradients(zip(gradients, trainable_variables))
print(gradients)
if gradients:
lstm_optim.apply_gradients(zip(gradients,lstm_model.trainable_variables))
else:
print('ERROR: no grads')
# LSTM reward is BERT_original_sentence - BERT_new_sentence
#   these are the difference in predictions between original and old
'''

model output time:  169.04650354385376
64 total logits
logprobs
64 [<tf.Tensor: shape=(), dtype=float32, numpy=-609.1123>, <tf.Tensor: shape=(), dtype=float32, numpy=-609.03296>, <tf.Tensor: shape=(), dtype=float32, numpy=-609.4209>, <tf.Tensor: shape=(), dtype=float32, numpy=-609.75146>, <tf.Tensor: shape=(), dtype=float32, numpy=-609.75806>, <tf.Tensor: shape=(), dtype=float32, numpy=-609.8074>, <tf.Tensor: shape=(), dtype=float32, numpy=-608.9331>, <tf.Tensor: shape=(), dtype=float32, numpy=-610.0338>, <tf.Tensor: shape=(), dtype=float32, numpy=-610.0483>, <tf.Tensor: shape=(), dtype=float32, numpy=-608.28174>, <tf.Tensor: shape=(), dtype=float32, numpy=-609.95245>, <tf.Tensor: shape=(), dtype=float32, numpy=-609.45886>, <tf.Tensor: shape=(), dtype=float32, numpy=-608.89087>, <tf.Tensor: shape=(), dtype=float32, numpy=-609.2401>, <tf.Tensor: shape=(), dtype=float32, numpy=-608.3739>, <tf.Tensor: shape=(), dtype=float32, numpy=-609.47534>, <tf.Tensor: shape=(), dtype=float32, numpy=-

"\n# diff is the baseline\n# logsum = tf.reduce_mean(logsum_batch * weight)\nlogsum = tf.reduce_sum(tf.nn.log_softmax(batch_logits[0]))\n# y = logsum * weight\n\n# print(st==base)\n# def compute_loss():\n# retrun -(classifier_model(base) - classifier_model(x))\n\n# lstm_optim.minimize(compute_loss,var_list = tf.constant([]))\n\n\ngradients = t.gradient(logsum, lstm_model.trainable_variables)\n# The optimize applies the update, using the variables\n# and the optimizer update rule\n# opt.apply_gradients(zip(gradients, trainable_variables))\nprint(gradients)\nif gradients:\nlstm_optim.apply_gradients(zip(gradients,lstm_model.trainable_variables))\nelse:\nprint('ERROR: no grads')\n# LSTM reward is BERT_original_sentence - BERT_new_sentence\n#   these are the difference in predictions between original and old\n"

In [ ]:
# BUG-- more strings than actions?
# POSSIBLE FIX-- use the Bert tokenizer for LSTM
print(len(splitstr))
print([x.numpy() for x in acts])
print(len(batch_logits[0]))

426
[0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,

In [ ]:
# use gradient tape object for updates

trainable_variables = [x]
gradients = t.gradient(y, trainable_variables)
# The optimize applies the update, using the variables
# and the optimizer update rule
opt.apply_gradients(zip(gradients, trainable_variables))

# sample = ['this is a sample sentence']
# # classifier_model(sample)
# classifier_model(tf.constant(sample))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.75665253]], dtype=float32)>

In [ ]:
# sess = tf.InteractiveSession()
# tf.compat.v1.disable_eager_execution()

# make core of policy network
obs_ph = tf.compat.v1.placeholder(shape=(None, len(batch)), dtype=tf.float32)
# logits = mlp(obs_ph, sizes=hidden_sizes+[n_acts])
logits = lstm_model(obs_ph)

# make action selection op (outputs int actions, sampled from policy)
actions = tf.squeeze(tf.multinomial(logits=logits,num_samples=1), axis=1)

# make loss function whose gradient, for the right data, is policy gradient
weights_ph = tf.placeholder(shape=(None,), dtype=tf.float32)
act_ph = tf.placeholder(shape=(None,), dtype=tf.int32)
action_masks = tf.one_hot(act_ph, n_acts)
log_probs = tf.reduce_sum(action_masks * tf.nn.log_softmax(logits), axis=1)
loss = -tf.reduce_mean(weights_ph * log_probs)


## MLP (BERT_tokens->{keep,delete})

In [ ]:
# alternative to LSTM
# every word is an individual training example
# 
# striclty worse than LSTM

def mlp(x, sizes, activation=tf.tanh, output_activation=None):
    # Build a feedforward neural network.
    for size in sizes[:-1]:
        x = tf.layers.dense(x, units=size, activation=activation)
    return tf.layers.dense(x, units=sizes[-1], activation=output_activation)

# input a one-hot of the word
mlp = Sequential()
mlp.add(Dense(1000, input_shape=input_shape, activation='relu'))
mlp.add(Dense(50, activation='relu'))
mlp.add(Dense(1, activation=None)